C:\Users\natha\Documents\School\FS22\CSE803\Project\py-sc\803-Final-Project\homography.py:28: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self.H = np.linalg.lstsq(x, y)[0]


In [19]:
    with open("./transformer.pickle", 'rb') as f:
        transformer = pickle.load(f)

In [20]:
transformer.x_scale

0.05

In [21]:
transformer.y_scale

0.05

In [13]:
x

array([[ 3.7407227 ,  1.7524414 , 11.988958  ],
       [ 2.7407227 ,  1.7526855 , 11.988958  ],
       [ 1.7409668 ,  1.7531738 , 11.988958  ],
       [ 0.74121094,  1.7539062 , 11.988958  ],
       [-0.25830078,  1.7546387 , 11.988958  ],
       [-1.2575684 ,  1.7558594 , 11.988958  ],
       [-1.2561035 ,  0.7573242 , 11.988958  ],
       [-1.2548828 , -0.24169922, 11.988958  ],
       [-1.2539062 , -1.2412109 , 11.988958  ],
       [-1.253418  , -2.2409668 , 11.988958  ],
       [-1.2529297 , -3.2407227 , 11.988958  ],
       [-1.2524414 , -4.2407227 , 11.988958  ]], dtype=float32)

In [6]:
t.homography_transform(x)

AttributeError: 'tuple' object has no attribute 'T'

In [25]:
y

array([[1.4355469 , 0.80859375],
       [1.3183594 , 0.80859375],
       [1.2011719 , 0.80859375],
       [1.0839844 , 0.80859375],
       [0.9667969 , 0.80859375],
       [0.8496094 , 0.80859375],
       [0.8496094 , 0.9160156 ],
       [0.8496094 , 1.0019531 ],
       [0.8496094 , 1.1132812 ],
       [0.8496094 , 1.2109375 ],
       [0.8496094 , 1.3203125 ],
       [0.8496094 , 1.4492188 ],
       [0.18945312, 0.10351562],
       [0.4609375 , 0.4296875 ],
       [0.41601562, 0.6269531 ],
       [0.8339844 , 0.08789062],
       [0.45507812, 0.83203125],
       [0.4453125 , 0.5175781 ],
       [0.5214844 , 0.83203125],
       [0.47070312, 0.43554688],
       [0.22070312, 0.2578125 ]], dtype=float32)

In [26]:
np.linalg.lstsq(x, y)

C:\Users\natha\AppData\Local\Temp\ipykernel_18752\1483668873.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  np.linalg.lstsq(x, y)


(array([[ 0.07231373, -0.03198319],
        [ 0.01374402, -0.05186034],
        [ 0.06692821,  0.06332152]], dtype=float32),
 array([1.3110583, 1.790185 ], dtype=float32),
 3,
 array([54.17134 , 18.383343, 11.046874], dtype=float32))